# SQL Aggregation and Join

![sql](img/sql-logo.jpg)

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("data/flights.db")
cur = conn.cursor()

# Objectives

- Use SQL aggregation functions with GROUP BY
- Use HAVING for group filtering
- Use SQL JOIN to combine tables using keys

# Aggregating Functions

>  A SQL **aggregating function** takes in many values and returns one value.

We have already seen some SQL aggregating functions like `COUNT()`. There are also others, like SUM(), AVG(), MIN(), and MAX().

## Example Simple Aggregations

In [2]:
# Max value for longitude
pd.read_sql('''
    SELECT 
        -- Note we have to cast to a numerical value first
        MAX(CAST(longitude AS REAL))
    FROM airports
''', conn)

,MAX(CAST(longitude AS REAL))
0,179.951


In [3]:
# Max value for id in table
pd.read_sql('''
SELECT 
    MAX(CAST(id AS integer))
FROM 
    airports
''', conn)

,MAX(CAST(id AS integer))
0,9541


In [4]:
# Effectively counts all the inactive airlines 
pd.read_sql('''
    SELECT COUNT()
    FROM airlines
    WHERE active='N'
''', conn)

,COUNT()
0,4886


We can also give aliases to our aggregations:

In [5]:
# Effectively counts all the active airlines 
pd.read_sql('''
    SELECT 
        COUNT() AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

,number_of_active_airlines
0,1161


# Grouping in SQL

We can go deeper and use aggregation functions on _groups_ using the `GROUP BY` clause.

The `GROUP BY` clause will group one or more columns together with the same values as one group to perform aggregation functions on.

## Example `GROUP BY`  Statements

Let's say we want to know how many active and non-active airlines there are.

### Without `GROUP BY`

Let's first start with just seeing how many airlines there are:

In [6]:
df_results = pd.read_sql('''
    SELECT 
        -- Reminder that this counts the number of rows before the SELECT
        COUNT() AS number_of_airlines
    FROM 
        airlines
''', conn)

df_results

,number_of_airlines
0,6048


One way for us to get the counts for each is to create two queries that will filter each kind of airline (active vs non-active) and count those values:

In [7]:
df_active = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

df_not_active = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_not_active_airlines
    FROM 
        airlines
    WHERE 
        active='N'
''', conn)

display(df_active)
display(df_not_active)

,number_of_active_airlines
0,1161


,number_of_not_active_airlines
0,4886


This works but it's inefficient.

### With `GROUP BY`

Instead, we can tell the SQL server to do the work for us by grouping values we care about for us!

In [8]:
df_results = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        active
''', conn)

df_results

,number_of_airlines
0,4886
1,1161
2,1


This is great! And if you look closely, you can observe we have _three_ different groups instead of our expected two!

Let's also print out the `airlines.active` value for each group/aggregation so we know what we're looking at:

In [9]:
df_results = pd.read_sql('''
    SELECT 
        airlines.active,
        COUNT() AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        airlines.active
''', conn)

df_results

,active,number_of_airlines
0,N,4886
1,Y,1161
2,n,1


## Group Task

- Which countries have the highest numbers of active airlines? Return the top 10.

In [ ]:
pd.read_sql('''
    SELECT
        *
    FROM
        airlines
''', conn)

<details>
    <summary><b>Possible Solution</b></summary>

``` sql
    pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
    LIMIT 10
    ''', conn)```
</details>

> Note that the `GROUP BY` clause is considered _before_ the `ORDER BY` and `LIMIT` clauses

## Exercise: Grouping

- Run a query that will return the number of airports by time zone. Each row should have a number of airports and a time zone.

In [ ]:
# Your code here


<details>
    <summary><b>Possible Solution</b></summary>

``` sql
pd.read_sql('''
    SELECT 
        airports.timezone
        ,COUNT() AS num_of_airports
    FROM 
        airports
    GROUP BY
        airports.timezone
    ORDER BY
        num_of_airports DESC
''', conn) ```
</details>

# Filtering Groups with `HAVING`

We showed that you can filter tables with `WHERE`. We can similarly filter _groups/aggregations_ using `HAVING` clauses.

## Examples of Using `HAVING`

### Simple Filtering - Number of Airports in a Country

Let's come back to the aggregation of active airports:

In [10]:
pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
''', conn)

,num,country
0,141,United States
1,72,Russia
2,40,United Kingdom
3,37,Germany
4,34,Canada
...,...,...
190,1,Syrian Arab Republic
191,1,Togo
192,1,Trinidad and Tobago
193,1,Turkmenistan


We can see we have a lot of results. But maybe we only want to keep the countries that have more than $30$ active airlines:

In [11]:
pd.read_sql('''
    SELECT 
        country,
        COUNT() AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING
        num > 30
    ORDER BY 
        num DESC
''', conn)

,country,num
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34


## Filtering Different Aggregations - Airport Altitudes

We can also filter on other aggregations. For example, let's say we want to investigate the `airports` table.

Specifically, we want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

### Looking at the `airports` Table

In [12]:
df_airports = pd.read_sql('''
    SELECT 
        *
    FROM 
        airports 
''', conn)

df_airports.head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


### Looking at the Highest Airport

Let's first get the highest altitude for each airport:

In [13]:
pd.read_sql('''
    SELECT 
        airports.country
        ,MAX(
            CAST(airports.altitude AS REAL)
        ) AS highest_airport_in_country
    FROM 
        airports 
    GROUP BY
        airports.country
    ORDER BY
        airports.country
''', conn)

,country,highest_airport_in_country
0,Afghanistan,7400.0
1,Albania,126.0
2,Algeria,4518.0
3,American Samoa,110.0
4,Angola,5778.0
...,...,...
235,West Bank,2485.0
236,Western Sahara,350.0
237,Yemen,7216.0
238,Zambia,4636.0


### Looking at the Number of Airports Too

We can also get the number of airports for each country.

In [14]:
pd.read_sql('''
    SELECT 
        airports.country
        ,MAX(
            CAST(airports.altitude AS REAL)
        ) AS highest_airport_in_country
        ,COUNT() AS number_of_airports_in_country
    FROM
        airports 
    GROUP BY
        airports.country
    ORDER BY
        airports.country
''', conn)

,country,highest_airport_in_country,number_of_airports_in_country
0,Afghanistan,7400.0,21
1,Albania,126.0,1
2,Algeria,4518.0,44
3,American Samoa,110.0,3
4,Angola,5778.0,26
...,...,...,...
235,West Bank,2485.0,1
236,Western Sahara,350.0,3
237,Yemen,7216.0,11
238,Zambia,4636.0,13


### Filtering on Aggregations

> Recall:
>
> We want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

In [16]:
pd.read_sql('''
    SELECT 
        airports.country
        ,MAX(
            CAST(airports.altitude AS REAL)
        ) AS highest_airport_in_country
        -- Note we don't have to include this in our SELECT
        ,COUNT() AS number_of_airports_in_country
    FROM
        airports 
    GROUP BY
        airports.country
    HAVING
        COUNT() >= 100
    ORDER BY
        airports.country
''', conn)

,country,highest_airport_in_country,number_of_airports_in_country
0,Argentina,11414.0,103
1,Australia,4260.0,263
2,Brazil,8708.0,213
3,Canada,3939.0,435
4,China,14472.0,219
5,France,6588.0,233
6,Germany,2297.0,321
7,India,10682.0,140
8,Indonesia,9104.0,114
9,Japan,2200.0,131


# Joins

The biggest advantage in using a relational database (like we've been with SQL) is that you can create **joins**.

> By using **`JOIN`** in our query, we can connect different tables using their _relationships_ to other tables.
>
> Usually we use a key (*foreign key*) to tell us how the two tables are related.

There are different types of joins and each has their different use case.

## `INNER JOIN`

> An **inner join** will join two tables together and only keep rows if the _key is in both tables_

![](img/inner_join.png)

Example of an inner join:

``` sql
SELECT
    table1.column_name,
    table2.different_column_name
FROM
    table1
    INNER JOIN table2
        ON table1.shared_column_name = table2.shared_column_name
```

### Code Example for Inner Joins

Let's say we want to look at the different airplane routes

In [17]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes 
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,None,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,None,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734


This is great but notice the `airline_id` column. It'd be nice to have some more information about the airlines associated with these routes.

We can do an **inner join** to get this information!

#### Inner Join Routes & Airline Data

In [18]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,index,id,name,alias,iata,icao,callsign,country,active
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,409,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66980,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3,4171,4178,Regional Express,\N,ZL,RXA,REX,Australia,Y
66981,67659,ZM,19016,DME,4029,FRU,2912,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66982,67660,ZM,19016,FRU,2912,DME,4029,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
66983,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734,5977,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y


We can also specify that we want to retain only certain columns in the `SELECT` clause:

In [19]:
pd.read_sql('''
    SELECT 
        routes.source AS departing
        ,routes.dest AS destination
        ,routes.stops AS stops_before_destination
        ,airlines.name AS airline
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

,departing,destination,stops_before_destination,airline
0,AER,KZN,0,Aerocondor
1,ASF,KZN,0,Aerocondor
2,ASF,MRV,0,Aerocondor
3,CEK,KZN,0,Aerocondor
4,CEK,OVB,0,Aerocondor
...,...,...,...,...
66980,WYA,ADL,0,Regional Express
66981,DME,FRU,0,Apache Air
66982,FRU,DME,0,Apache Air
66983,FRU,OSS,0,Apache Air


#### Note: Losing Data with Inner Joins

Since data rows are kept only if _both_ tables have the key, some data can be lost

In [20]:
df_all_routes = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
''', conn)

df_routes_after_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

In [21]:
# Look at how the number of rows are different
df_all_routes.shape, df_routes_after_join.shape

((67663, 10), (66985, 19))

If you want to keep your data from at least one of your tables, you should use a left join instead of an inner join.

## `LEFT JOIN`

> A **left join** will join two tables together and but will keep all data from the first (left) table using the key provided.

![](img/left_join.png)

Example of a left and right join:

```sql
SELECT
    table1.column_name,
    table2.different_column_name
FROM
    table1
    LEFT JOIN table2
        ON table1.shared_column_name = table2.shared_column_name
```

### Code Example for Left Join

Recall our example using an inner join and how it lost some data since the key wasn't in both the `routes` _and_ `airlines` tables. 

In [22]:
df_all_routes = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
''', conn)

# This will lose some data (some routes not included)
df_routes_after_inner_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        INNER JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

# The number of rows are different
df_all_routes.shape, df_routes_after_inner_join.shape

((67663, 10), (66985, 19))

If wanted to ensure we always had every route even if the key in `airlines` was not found, we could replace our `INNER JOIN` with a `LEFT JOIN`:

In [23]:
# This will include all the data from routes
df_routes_after_left_join = pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        LEFT JOIN airlines
            ON routes.airline_id = airlines.id
''', conn)

df_routes_after_left_join.shape

(67663, 19)

## Exercise: Joins

Which airline has the most routes listed in our database?

In [ ]:
# Your code here


<details>
    <summary><b>Possible Solution</b></summary>

```sql
SELECT
    airlines.name AS airline,
    COUNT() AS number_of_routes
-- We first need to get all the relevant info via a join
FROM
    routes
    -- LEFT JOIN since we want all routes (even if airline id is unknown)
    LEFT JOIN airlines
        ON routes.airline_id = airlines.id
-- We need to group by airline's ID
GROUP BY
    airlines.id
ORDER BY
    number_of_routes DESC
```
</details>

# Level Up: Execution Order

```SQL
SELECT 
    COUNT(table2.col2) AS my_new_count
    ,table1.col2
FROM
    table1
    JOIN table2
        ON table1.col1 = table2.col2
WHERE
    table1.col1 > 0
GROUP BY
    table2.col1
```

1. `From`
2. `Where`
3. `Group By`
4. `Having`
5. `Select`
6. `Order By`
7. `Limit`